In [1]:
import dash
import pandas as pd
import plotly.express as px
from dash import html
from dash import dcc
from dash.dependencies import Input, Output

In [2]:
df = pd.read_csv('cc_data.csv')
#d = ['2021-12-08','2021-12-07','2021-12-06']
#dft = df[df['Date'].isin(d)]
l = [0,1,3,4,5]
dfM = df.iloc[:,l]
dfM = pd.melt(dfM, id_vars=['Date','Date_Time','Diff'],var_name='hash', value_name='rate')
#dfM[dfM['Date']=='2021-12-24']
#df.groupby(by='Date').mean()['Diff'][-1:]
dfM
#df.groupby(by='Date').mean()[-1:]['Calculated_Hash']

,Date,Date_Time,Diff,hash,rate
0,0,0,0.000000,Reported_Hash,0
1,2021-11-21,13:33:51.397040,0.000000,Reported_Hash,30
2,2021-11-21,13:34:53.458088,0.000000,Reported_Hash,30
3,2021-11-21,14:01:40.669087,350.001428,Reported_Hash,30
4,2021-11-21,14:03:51.583825,141.313850,Reported_Hash,30
...,...,...,...,...,...
2339,2021-12-27,22:28:08.003713,942.026138,Calculated_Hash,24
2340,2021-12-27,22:43:18.500120,652.458966,Calculated_Hash,29
2341,2021-12-27,22:58:30.300014,829.143777,Calculated_Hash,24
2342,2021-12-27,23:13:41.143457,525.223814,Calculated_Hash,19


In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div([ #main div
    html.Div(children=[ #Graph, Dropdown and Radio Div
    dcc.Graph(id='graph-with-drop'), 
    dcc.Dropdown(
        id='drop-year',
        options=[{'label':i,'value':i} for i in dfM['Date'].unique()],value=dfM['Date'].max()
    ), 
    dcc.RadioItems(
        id='radio_coin_hash',
        options=[
            {'label':'Hash Rate','value':'rate'},
            {'label':'Coins','value':'Diff'}
        ],value='rate'#CLOSE RADIO list
    ), #CLOSE RADIO button
        html.Br(),
        ]), #CLOSE GRAPH, DROPDOWN and RADIO Div
    
#----Pie  
    
    html.Div(children=[ #Average hash pie div
        html.H1('Average Hash Rate'),
        dcc.Graph(id='average_hash_pie')
    ],style={'padding': 10, 'flex': 1, 'display': 'inline-block', 'margin-left':'75px'}), #CLOSE AVERAGE HASH div
    
    html.Div(children=[ #reported hash pie div
        html.H1('Current Hash Rate'),
        dcc.Graph(id='reported_hash_pie')
    ],style={'padding': 10, 'flex': 1, 'display': 'inline-block', 'margin-left':'75px'}), #CLOSE REPORTED HASH div
    
    html.Div(children=[ #calc hash pie div
        html.H1('Calculated Hash Rate'),
        dcc.Graph(id='calc_hash_pie')
    ],style={'padding': 10, 'flex': 1, 'display': 'inline-block', 'margin-left':'75px'}), #CLOSE CALC HASH div
    
    html.Br(),

#---Coin    
    html.Div(children=[ #Average coin div
        html.H2('Day_Avg_Coin_Per_Hr'),
        html.Div(id='avg_coin_hour')
    ],style={'padding': 10, 'flex': 1, 'display': 'inline-block', 'margin-left':'75px'}), #CLOSE AVERAGE COIN div
    
    html.Div(children=[ #Days Coin div
        html.H2('Total_Day_Coin'),
        html.Div(title='Todays Coin',id='day_tot_coin')
    ], style={'padding': 10, 'flex': 1, 'display': 'inline-block', 'margin-left':'75px'}), #CLOSE TOTAL COIN div
    
    html.Div(children=[ #Total Coin div
        html.H2('Total_Coin'),
        html.Div(title='Total Coin',id='tot_coin')
    ], style={'padding': 10, 'flex': 1, 'display': 'inline-block', 'margin-left':'75px'}) #CLOSE TOTAL COIN div
    
]) #close main div

#---Graph call back

@app.callback( #Callback for graph and radio button
    Output('graph-with-drop', 'figure'),
    Input('drop-year', 'value'),
    Input('radio_coin_hash', 'value'))
def update_figure(selected_year,cat):
    filtered_df = dfM[dfM.Date == selected_year]
    fig = px.bar(filtered_df, x="Date_Time", y=cat,
                    color="hash", barmode='group')
    fig.update_layout(transition_duration=500)
    return fig

#----Pie callback

@app.callback( #Call back for the average rate pie
    Output('average_hash_pie', 'figure'),
    Input('drop-year', 'value'))
def update_figure(selected_year):
    ar=df.groupby(by='Date').mean()[-1:]['Calculated_Hash']

    labels = pd.Series([int(ar),'..'])
    values = pd.Series([int(ar),100-int(ar)])

    f = {'Label':labels, 'Value':values}
    dfg = pd.DataFrame(f)

    fig = px.pie(dfg,values='Value',names='Label')
    fig.update_traces(textinfo='label', hole=.5, textfont_size=18)
    fig.update_layout(annotations=[dict(text='Average_Hash_Rate',showarrow=False, font_size=20)]) 
    return fig

@app.callback( #Call back for the reported hash rate pie
    Output('reported_hash_pie', 'figure'),
    Input('drop-year', 'value'))
def update_figure(selected_year):
    dfRH = dfM[(dfM['Date']==dfM['Date'].max()) & (dfM['hash']=='Reported_Hash')]
    rh = dfRH[dfRH['Date_Time']==dfRH['Date_Time'].max()]['rate']

    labels = pd.Series([int(rh),'..'])
    values = pd.Series([int(rh),100-int(rh)])

    f = {'Label':labels, 'Value':values}
    df = pd.DataFrame(f)

    fig = px.pie(df,values='Value',names='Label')
    fig.update_traces(textinfo='label', hole=.5, textfont_size=18)
    fig.update_layout(annotations=[dict(text='Current_Hash_Rate',showarrow=False, font_size=20)]) 
    return fig

@app.callback( #Call back for the reported calc rate pie
    Output('calc_hash_pie', 'figure'),
    Input('drop-year', 'value'))
def update_figure(selected_year):
    dfRH = dfM[(dfM['Date']==dfM['Date'].max()) & (dfM['hash']=='Calculated_Hash')]
    ch = dfRH[dfRH['Date_Time']==dfRH['Date_Time'].max()]['rate']

    labels = pd.Series([int(ch),'..'])
    values = pd.Series([int(ch),100-int(ch)])

    f = {'Label':labels, 'Value':values}
    df = pd.DataFrame(f)

    fig = px.pie(df,values='Value',names='Label')
    fig.update_traces(textinfo='label', hole=.5, textfont_size=18)
    fig.update_layout(annotations=[dict(text='Calculated_Hash_Rate',showarrow=False, font_size=20)]) 
    return fig

#---Coin Calc

@app.callback( #Callback for Total Coins
    Output('avg_coin_hour', component_property='children'),
    Input('drop-year', 'value'))
def update_tot_coin(selected_year):
    a=df.groupby(by='Date').mean()['Diff'][-1:]
    return a

@app.callback( #Callback for Total Coins
    Output('day_tot_coin', component_property='children'),
    Input('drop-year', 'value'))
def update_tot_coin(selected_year):
    d=df.groupby(by='Date').sum()['Diff'][-1:]
    return d

@app.callback( #Callback for Total Coins
    Output('tot_coin', component_property='children'),
    Input('drop-year', 'value'))
def update_tot_coin(selected_year):
    t=df[df['Date']==selected_year][-1:]['Num_Coins']
    return t

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Dec/2021 23:31:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2021 23:31:47] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2021 23:31:47] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2021 23:31:47] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Dec/2021 23:31:47] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Dec/2021 23:31:47] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [27/Dec/2021 23:31:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2021 23:31:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2021 23:31:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2021 23:31:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [27/Dec/2021 23:31:48] "POST /_dash-update-component HTTP/1.

In [ ]:
#Current_Hash=dfM['rate'][-1:]
#Current_Hash

#dfRH = dfM[(dfM['Date']==dfM['Date'].max()) & (dfM['hash']=='Reported_Hash')]
#rh = dfRH[dfRH['Date_Time']==dfRH['Date_Time'].max()]['rate']
#dfM[(dfM['Date_Time']==dfM['Date_Time'].max()) & (dfM['Date']==dfM['Date'].max())]
#dfM['Date_Time']==dfM['Date_Time'].max()